In [1]:
#Putting all the import packages over here.
import pandas as pd
import re
pd.options.display.max_columns = 30  #Can't have too many columns.
import glob
from datetime import datetime
import dateutil.parser
import nltk
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize']= (17,5)
from collections import Counter
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.stem.porter import PorterStemmer
from sklearn.cluster import KMeans
import langid
from tqdm import tqdm
import os

In [2]:
#Checking if all the speeches are loaded or not
folder = nltk.data.find('A:/Data Analysis Jupyter/Campaign-Analytics/Speeches/narendramodi/2018/')
path_link = 'A:/Data Analysis Jupyter/Campaign-Analytics/Speeches/narendramodi/2018/*'
paths = glob.glob(path_link)
len(paths)

44

In [5]:
x = "01_06_2018_PM’s Keynote Address at Shangri La Dialogue"
date = re.search(r"(\d\d)_(\d\d)_(\d\d\d\d)", x)
date = datetime.strptime(date.group(0), '%d_%m_%Y')
date


datetime.datetime(2018, 6, 1, 0, 0)

In [6]:
nm_speeches = [ ]  #Empty dataframe to store all the content of speeches
for path in paths:
    corpusReader = nltk.corpus.PlaintextCorpusReader(folder, '.*\.txt')
    # opening the files and converting it to a dictionary
    with open(path, encoding="utf-8") as speech_file:
        filename = path.split('/')[-1]
        date = re.search(r"(\d\d)_(\d\d)_(\d\d\d\d)", filename) 
        if date:
            date = datetime.strptime(date.group(0), '%d_%m_%Y')
        else:
            date = np.nan
        speech = {
            'Title' : path.split('/')[-1],
            'Speech' : speech_file.read(),
            'Date' : date
        }   
    nm_speeches.append(speech)
nm_df = pd.DataFrame(nm_speeches)
nm_df.head(10)

,Date,Speech,Title
0,2018-06-01,"Prime Minister Lee Hsien Loong,Thank you for y...",2018\01_06_2018_PM’s Keynote Address at Shangr...
1,2018-06-01,His Excellency \nPrime Minister \nLee Sean Loo...,2018\01_06_2018_PM’s Press Statement during hi...
2,2018-09-01,All the distinguished dignitaries present here...,2018\01_09_2018_PM’s address at the Outstandin...
3,2018-11-02,"In the Cabinet, my colleagues Arun Jaitley, Gi...",2018\02_11_2018_PM’s address at the launch of ...
4,2018-06-05,"My Ministerial Colleagues Dr. Harsh Vardhan, D...",2018\05_06_2018_PM’s address on the occasion o...
5,2018-06-05,"My dear brothers and sisters, helloI have alwa...",2018\05_06_2018_PM’s address to the beneficiar...
6,2018-09-09,"Hon. Mr. Chairman,First of all, I congratulate...",2018\09_09_2018_PM’s address in Parliament on ...
7,2018-09-10,All the ladies and gentlemen associated with d...,2018\10_09_2018_PM’s address at the inaugurati...
8,2018-09-11,"Today is August 11. 110 years ago today, for t...",2018\11_09_2018_PM’s address at the 56th Annua...
9,2018-06-14,"Bharat Mata Ki Jai, Bhilai Steel Plant is the ...",2018\14_06_2018_PM’s address at the launch of ...


In [7]:
nm_df.head()

,Date,Speech,Title
0,2018-06-01,"Prime Minister Lee Hsien Loong,Thank you for y...",2018\01_06_2018_PM’s Keynote Address at Shangr...
1,2018-06-01,His Excellency \nPrime Minister \nLee Sean Loo...,2018\01_06_2018_PM’s Press Statement during hi...
2,2018-09-01,All the distinguished dignitaries present here...,2018\01_09_2018_PM’s address at the Outstandin...
3,2018-11-02,"In the Cabinet, my colleagues Arun Jaitley, Gi...",2018\02_11_2018_PM’s address at the launch of ...
4,2018-06-05,"My Ministerial Colleagues Dr. Harsh Vardhan, D...",2018\05_06_2018_PM’s address on the occasion o...


In [13]:
nm_df['Year'],nm_df['Month'],nm_df['Day'] = nm_df.Date.dt.year, nm_df.Date.dt.month, nm_df.Date.dt.day

In [14]:
nm_df.head()

,Date,Speech,Title,year,month,day,Year,Month,Day
0,2018-06-01,"Prime Minister Lee Hsien Loong,Thank you for y...",2018\01_06_2018_PM’s Keynote Address at Shangr...,2018,6,1,2018,6,1
1,2018-06-01,His Excellency \nPrime Minister \nLee Sean Loo...,2018\01_06_2018_PM’s Press Statement during hi...,2018,6,1,2018,6,1
2,2018-09-01,All the distinguished dignitaries present here...,2018\01_09_2018_PM’s address at the Outstandin...,2018,9,1,2018,9,1
3,2018-11-02,"In the Cabinet, my colleagues Arun Jaitley, Gi...",2018\02_11_2018_PM’s address at the launch of ...,2018,11,2,2018,11,2
4,2018-06-05,"My Ministerial Colleagues Dr. Harsh Vardhan, D...",2018\05_06_2018_PM’s address on the occasion o...,2018,6,5,2018,6,5


In [15]:
nm_df = nm_df[["Year","Month", "Day", "Title", "Speech", "Date"]]

In [18]:
nm_df.head()

,Year,Month,Day,Title,Speech,Date
0,2018,6,1,2018\01_06_2018_PM’s Keynote Address at Shangr...,"Prime Minister Lee Hsien Loong,Thank you for y...",2018-06-01
1,2018,6,1,2018\01_06_2018_PM’s Press Statement during hi...,His Excellency \nPrime Minister \nLee Sean Loo...,2018-06-01
2,2018,9,1,2018\01_09_2018_PM’s address at the Outstandin...,All the distinguished dignitaries present here...,2018-09-01
3,2018,11,2,2018\02_11_2018_PM’s address at the launch of ...,"In the Cabinet, my colleagues Arun Jaitley, Gi...",2018-11-02
4,2018,6,5,2018\05_06_2018_PM’s address on the occasion o...,"My Ministerial Colleagues Dr. Harsh Vardhan, D...",2018-06-05


In [19]:
nm_df.to_pickle('speeches_df_2018.pkl')